# 🗑️ Détection et Classification de Poubelles avec YOLO

Ce notebook implémente un système complet de détection et classification de poubelles en utilisant **uniquement YOLOv8** :
- **YOLOv8** : Détection des poubelles ET classification (plein/vide) en une seule étape

Plus simple, plus rapide et plus efficace qu'utiliser deux modèles séparés !
Toutes les fonctions sont en français.

## 1. Installation et Importation des Bibliothèques

In [5]:
# Installation des bibliothèques nécessaires
%pip install --user ultralytics opencv-python pillow matplotlib numpy pandas pyyaml roboflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Importation des bibliothèques
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import yaml
import os
from PIL import Image

print("✓ Bibliothèques importées avec succès!")

✓ Bibliothèques importées avec succès!


## 2. Configuration du Projet

## 3. Téléchargement du Dataset depuis Roboflow

In [7]:
# Configuration
CONFIG = {
    # Chemins (seront mis à jour après téléchargement Roboflow)
    'dossier_dataset': '',  # Sera rempli par Roboflow
    'yaml_path': '',         # Sera rempli par Roboflow
    'poids_yolo': 'models/yolo_poubelles.pt',
    'dossier_resultats': 'results',
    
    # Paramètres YOLO
    'modele_base': 'yolov8n.pt',  # n=nano (rapide), s=small, m=medium, l=large
    'taille_image': 640,
    'epochs': 30,
    'batch_size': 16,
    'confiance': 0.5,
    'iou': 0.45,
    
    # Roboflow
    'roboflow_api_key': '4zGCDGgIjB5Lg8TVVqSY',
    'roboflow_workspace': 'deep-nhhm8',
    'roboflow_project': 'my-first-project-prs1r',
    'roboflow_version': 2
}

print("✓ Configuration chargée")
print("📊 Dataset sera téléchargé depuis Roboflow")

✓ Configuration chargée
📊 Dataset sera téléchargé depuis Roboflow


In [8]:
from roboflow import Roboflow

def telecharger_dataset_roboflow():
    """
    Télécharge le dataset depuis Roboflow
    """
    print("📥 Téléchargement du dataset depuis Roboflow...")
    print("="*70)
    
    # Initialiser Roboflow avec les paramètres du CONFIG
    rf = Roboflow(api_key=CONFIG['roboflow_api_key'])
    project = rf.workspace(CONFIG['roboflow_workspace']).project(CONFIG['roboflow_project'])
    version = project.version(CONFIG['roboflow_version'])
    
    # Télécharger au format YOLOv9 (compatible avec YOLOv8)
    dataset = version.download("yolov9")
    
    print(f"\n✓ Dataset téléchargé dans: {dataset.location}")
    print(f"✓ Fichier data.yaml: {dataset.location}/data.yaml")
    
    # Mettre à jour la configuration
    CONFIG['dossier_dataset'] = dataset.location
    CONFIG['yaml_path'] = f"{dataset.location}/data.yaml"
    
    print("\n📊 Informations du dataset:")
    print(f"  - Localisation: {dataset.location}")
    print(f"  - Format: YOLOv9/YOLOv8")
    print("="*70)
    
    return dataset


# Télécharger le dataset
dataset = telecharger_dataset_roboflow()

📥 Téléchargement du dataset depuis Roboflow...
loading Roboflow workspace...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow project...

✓ Dataset téléchargé dans: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2
✓ Fichier data.yaml: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2/data.yaml

📊 Informations du dataset:
  - Localisation: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2
  - Format: YOLOv9/YOLOv8

✓ Dataset téléchargé dans: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2
✓ Fichier data.yaml: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2/data.yaml

📊 Informations du dataset:
  - Localisation: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2
  - Format: YOLOv9/YOLOv8


## 4. Vérification du Dataset

In [9]:
def verifier_dataset():
    """
    Vérifie que le dataset Roboflow est bien téléchargé et configuré
    """
    if not CONFIG['dossier_dataset']:
        print("❌ Dataset non téléchargé. Exécutez d'abord la cellule de téléchargement Roboflow!")
        return False
    
    dataset_path = Path(CONFIG['dossier_dataset'])
    yaml_path = Path(CONFIG['yaml_path'])
    
    print("🔍 Vérification du dataset...")
    print("="*70)
    
    # Vérifier le fichier YAML
    if yaml_path.exists():
        print(f"✓ Fichier data.yaml trouvé: {yaml_path}")
        
        # Lire le contenu
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)
            print(f"\n📊 Contenu du data.yaml:")
            print(f"  - Nombre de classes: {data.get('nc', 'N/A')}")
            print(f"  - Classes: {data.get('names', 'N/A')}")
            print(f"  - Train: {data.get('train', 'N/A')}")
            print(f"  - Val: {data.get('val', 'N/A')}")
            if 'test' in data:
                print(f"  - Test: {data.get('test', 'N/A')}")
    else:
        print(f"❌ Fichier data.yaml non trouvé: {yaml_path}")
        return False
    
    # Compter les images
    train_images = list(dataset_path.glob('**/train/images/*.*'))
    val_images = list(dataset_path.glob('**/val/images/*.*'))
    test_images = list(dataset_path.glob('**/test/images/*.*'))
    
    print(f"\n📁 Statistiques du dataset:")
    print(f"  - Images d'entraînement: {len(train_images)}")
    print(f"  - Images de validation: {len(val_images)}")
    print(f"  - Images de test: {len(test_images)}")
    print(f"  - Total: {len(train_images) + len(val_images) + len(test_images)}")
    
    print("="*70)
    print("✓ Dataset prêt pour l'entraînement!")
    
    return True


# Vérifier le dataset (décommentez après avoir téléchargé)
verifier_dataset()

🔍 Vérification du dataset...
✓ Fichier data.yaml trouvé: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2\data.yaml

📊 Contenu du data.yaml:
  - Nombre de classes: 2
  - Classes: ['poubelle_pleine', 'poubelle_vide']
  - Train: ../train/images
  - Val: ../valid/images
  - Test: ../test/images

📁 Statistiques du dataset:
  - Images d'entraînement: 345
  - Images de validation: 0
  - Images de test: 25
  - Total: 370
✓ Dataset prêt pour l'entraînement!


True

In [10]:
# Appeler la vérification
verifier_dataset()

🔍 Vérification du dataset...
✓ Fichier data.yaml trouvé: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2\data.yaml

📊 Contenu du data.yaml:
  - Nombre de classes: 2
  - Classes: ['poubelle_pleine', 'poubelle_vide']
  - Train: ../train/images
  - Val: ../valid/images
  - Test: ../test/images

📁 Statistiques du dataset:
  - Images d'entraînement: 345
  - Images de validation: 0
  - Images de test: 25
  - Total: 370
✓ Dataset prêt pour l'entraînement!


True

## 5. Entraînement du Modèle YOLO

In [11]:
def entrainer_yolo():
    """
    Entraîne YOLOv8 sur le dataset Roboflow
    Détection + Classification en une seule étape !
    """
    if not CONFIG['yaml_path']:
        print("❌ Erreur: Téléchargez d'abord le dataset depuis Roboflow!")
        return None
    
    print("\n" + "="*70)
    print("ENTRAÎNEMENT YOLO")
    print("Dataset: Roboflow")
    print("Détection + Classification en une seule étape")
    print("="*70 + "\n")
    
    # Charger le modèle de base pré-entraîné
    modele = YOLO(CONFIG['modele_base'])
    
    print(f"✓ Modèle {CONFIG['modele_base']} chargé")
    print(f"✓ Dataset YAML: {CONFIG['yaml_path']}")
    print(f"\nDébut de l'entraînement...\n")
    
    # Entraîner le modèle
    resultats = modele.train(
        data=CONFIG['yaml_path'],
        epochs=CONFIG['epochs'],
        imgsz=CONFIG['taille_image'],
        batch=CONFIG['batch_size'],
        name='detection_classification_poubelles',
        project='runs/detect',
        patience=15,
        save=True,
        plots=True,
        verbose=True,
        device='cpu'  # 'cpu' car pas de GPU disponible
    )
    
    print("\n" + "="*70)
    print("✓ ENTRAÎNEMENT TERMINÉ!")
    print("="*70)
    
    # Sauvegarder le meilleur modèle
    chemin_best = Path('runs/detect/detection_classification_poubelles/weights/best.pt')
    if chemin_best.exists():
        import shutil
        Path(CONFIG['poids_yolo']).parent.mkdir(exist_ok=True, parents=True)
        shutil.copy(chemin_best, CONFIG['poids_yolo'])
        print(f"\n✓ Meilleur modèle sauvegardé: {CONFIG['poids_yolo']}")
    
    return modele


# Note: Décommentez pour lancer l'entraînement après avoir téléchargé le dataset
modele_yolo = entrainer_yolo()


ENTRAÎNEMENT YOLO
Dataset: Roboflow
Détection + Classification en une seule étape

✓ Modèle yolov8n.pt chargé
✓ Dataset YAML: c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2/data.yaml

Début de l'entraînement...

Ultralytics 8.3.230  Python-3.10.18 torch-2.9.1+cpu CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
Ultralytics 8.3.230  Python-3.10.18 torch-2.9.1+cpu CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Dell\Documents\DEEP LEARNING\Nouveau dossier\My-First-Project-2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freez

KeyboardInterrupt: 

In [12]:
# Charger votre modèle entraîné depuis Google Colab
import os

# Vérifier si le modèle existe
if os.path.exists(CONFIG['poids_yolo']):
    modele = YOLO(CONFIG['poids_yolo'])
    print(f"✅ Modèle chargé avec succès : {CONFIG['poids_yolo']}")
    print(f"Classes du modèle : {modele.names}")
else:
    print(f"❌ Modèle non trouvé : {CONFIG['poids_yolo']}")
    print("\n📋 Instructions :")
    print("1. Copiez le fichier 'best.pt' depuis Google Colab")
    print("2. Créez le dossier 'models/' si nécessaire")
    print("3. Placez le fichier dans 'models/yolo_poubelles.pt'")

✅ Modèle chargé avec succès : models/yolo_poubelles.pt
Classes du modèle : {0: 'poubelle_pleine', 1: 'poubelle_vide'}


### 📥 Charger le modèle déjà entraîné

Si vous avez déjà entraîné le modèle sur Google Colab :
1. Placez le fichier `best.pt` dans le dossier `models/` 
2. Renommez-le en `yolo_poubelles.pt`
3. Exécutez la cellule suivante pour charger le modèle

## 6. Validation du Modèle

In [ ]:
def valider_modele():
    """
    Valide le modèle YOLO sur les données de test
    """
    chemin_modele = CONFIG['poids_yolo']
    
    if not Path(chemin_modele).exists():
        print(f"❌ Modèle non trouvé: {chemin_modele}")
        print("Entraînez d'abord le modèle!")
        return None
    
    if not CONFIG['yaml_path']:
        print("❌ Erreur: Chemin du dataset YAML non configuré!")
        return None
    
    print("Chargement du modèle...")
    modele = YOLO(chemin_modele)
    
    print("Validation en cours...\n")
    
    # Valider sur le dataset
    resultats = modele.val(
        data=CONFIG['yaml_path'],
        split='test'
    )
    
    print("\n" + "="*70)
    print("📊 RÉSULTATS DE VALIDATION")
    print("="*70)
    print(f"mAP50: {resultats.box.map50:.4f}")
    print(f"mAP50-95: {resultats.box.map:.4f}")
    print(f"Précision: {resultats.box.mp:.4f}")
    print(f"Rappel: {resultats.box.mr:.4f}")
    print("="*70)
    
    return resultats


# Note: Décommentez après l'entraînement
# resultats_validation = valider_modele()

## 7. Détection et Classification sur Images

In [ ]:
def charger_modele_entraine():
    """
    Charge le modèle YOLO entraîné
    """
    chemin_modele = CONFIG['poids_yolo']
    
    if not Path(chemin_modele).exists():
        print(f"⚠️ Modèle personnalisé non trouvé")
        print(f"Utilisation du modèle de base: {CONFIG['modele_base']}")
        return YOLO(CONFIG['modele_base'])
    
    modele = YOLO(chemin_modele)
    print(f"✓ Modèle chargé: {chemin_modele}")
    return modele


def detecter_et_classifier(image_path, modele, seuil_confiance=0.5):
    """
    Détecte et classifie les poubelles en une seule étape avec YOLO
    
    Args:
        image_path: Chemin vers l'image
        modele: Modèle YOLO
        seuil_confiance: Seuil de confiance minimum
    
    Returns:
        image_resultat: Image annotée
        detections: Liste des détections avec classes
    """
    # Charger l'image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"❌ Erreur: Impossible de charger {image_path}")
        return None, []
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Prédiction avec YOLO (détection + classification)
    resultats = modele(image, conf=seuil_confiance, verbose=False)
    
    detections = []
    
    for resultat in resultats:
        boxes = resultat.boxes
        
        for box in boxes:
            # Extraire les informations
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            confiance = float(box.conf[0].cpu().numpy())
            classe_id = int(box.cls[0].cpu().numpy())
            nom_classe = modele.names[classe_id]
            
            # Déterminer l'état (vide ou plein)
            etat = 'vide' if 'vide' in nom_classe.lower() or 'empty' in nom_classe.lower() else 'plein'
            
            # Stocker la détection
            detections.append({
                'bbox': (x1, y1, x2, y2),
                'confiance': confiance,
                'classe': nom_classe,
                'etat': etat
            })
            
            # Dessiner sur l'image
            couleur = (0, 255, 0) if etat == 'vide' else (255, 0, 0)
            cv2.rectangle(image_rgb, (x1, y1), (x2, y2), couleur, 3)
            
            # Texte
            texte = f"{etat.upper()} ({confiance:.2f})"
            
            # Fond pour le texte
            (text_width, text_height), _ = cv2.getTextSize(
                texte, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2
            )
            cv2.rectangle(
                image_rgb, 
                (x1, y1 - text_height - 10), 
                (x1 + text_width, y1), 
                couleur, 
                -1
            )
            
            cv2.putText(
                image_rgb, texte, (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2
            )
    
    return image_rgb, detections


def afficher_resultats(image_resultat, detections):
    """
    Affiche les résultats de détection et classification
    """
    if image_resultat is None:
        return
    
    # Afficher l'image
    plt.figure(figsize=(12, 8))
    plt.imshow(image_resultat)
    plt.axis('off')
    plt.title(f'Détections: {len(detections)} poubelle(s)', 
              fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Afficher les détails
    if len(detections) == 0:
        print("\n⚠️ Aucune poubelle détectée")
        return
    
    print("\n" + "="*70)
    print("📊 RÉSULTATS DE DÉTECTION ET CLASSIFICATION")
    print("="*70)
    
    for i, det in enumerate(detections, 1):
        icone = '✅' if det['etat'] == 'vide' else '🔴'
        print(f"\nPoubelle {i}:")
        print(f"  {icone} État: {det['etat'].upper()}")
        print(f"  📍 Confiance: {det['confiance']:.1%}")
        print(f"  📦 Position: {det['bbox']}")
    
    # Statistiques
    nb_vides = sum(1 for d in detections if d['etat'] == 'vide')
    nb_pleines = sum(1 for d in detections if d['etat'] == 'plein')
    
    print("\n" + "="*70)
    print("📈 STATISTIQUES")
    print("="*70)
    print(f"  Total: {len(detections)} poubelle(s)")
    print(f"  Vides: {nb_vides}")
    print(f"  Pleines: {nb_pleines}")
    print("="*70)


# Exemple d'utilisation
# modele = charger_modele_entraine()
# image_resultat, detections = detecter_et_classifier('chemin/vers/image.jpg', modele)
# afficher_resultats(image_resultat, detections)

## 8. Détection sur Vidéo ou Webcam

In [ ]:
def detecter_video(source, modele, afficher=True, sauvegarder=False, chemin_sortie='output.mp4'):
    """
    Détecte et classifie les poubelles dans une vidéo ou webcam
    
    Args:
        source: Chemin de la vidéo ou 0 pour webcam
        modele: Modèle YOLO
        afficher: Afficher la vidéo en temps réel
        sauvegarder: Sauvegarder la vidéo annotée
        chemin_sortie: Chemin de sauvegarde
    """
    cap = cv2.VideoCapture(source)
    
    if not cap.isOpened():
        print(f"❌ Erreur: Impossible d'ouvrir {source}")
        return
    
    # Configuration pour la sauvegarde
    if sauvegarder:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter(chemin_sortie, fourcc, fps, (width, height))
    
    print("📹 Traitement vidéo en cours...")
    print("Appuyez sur 'q' pour quitter")
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Détection avec YOLO
        resultats = modele(frame, conf=CONFIG['confiance'], verbose=False)
        
        # Annoter le frame
        frame_annote = resultats[0].plot()
        
        # Afficher
        if afficher:
            cv2.imshow('Détection Poubelles', frame_annote)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Sauvegarder
        if sauvegarder:
            out.write(frame_annote)
        
        if frame_count % 30 == 0:
            print(f"  Frame {frame_count} traité...")
    
    cap.release()
    if sauvegarder:
        out.release()
        print(f"\n✓ Vidéo sauvegardée: {chemin_sortie}")
    
    if afficher:
        cv2.destroyAllWindows()
    
    print(f"\n✓ {frame_count} frames traités")


# Exemples d'utilisation
# modele = charger_modele_entraine()

# Webcam
# detecter_video(0, modele)

# Vidéo
# detecter_video('chemin/vers/video.mp4', modele, sauvegarder=True)

## 9. Détection sur Multiple Images (Batch)

In [ ]:
def detecter_dossier(dossier_images, modele, sauvegarder=True):
    """
    Détecte et classifie les poubelles pour toutes les images d'un dossier
    
    Args:
        dossier_images: Chemin du dossier contenant les images
        modele: Modèle YOLO
        sauvegarder: Sauvegarder les images annotées
    """
    dossier = Path(dossier_images)
    
    if not dossier.exists():
        print(f"❌ Dossier non trouvé: {dossier}")
        return
    
    # Trouver toutes les images
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    images = []
    for ext in extensions:
        images.extend(dossier.glob(ext))
        images.extend(dossier.glob(ext.upper()))
    
    if len(images) == 0:
        print(f"⚠️ Aucune image trouvée dans {dossier}")
        return
    
    print(f"📁 {len(images)} image(s) trouvée(s)")
    print("Traitement en cours...\n")
    
    # Dossier de sortie
    if sauvegarder:
        dossier_sortie = dossier / 'resultats'
        dossier_sortie.mkdir(exist_ok=True)
    
    resultats_globaux = []
    
    for i, image_path in enumerate(images, 1):
        print(f"[{i}/{len(images)}] {image_path.name}")
        
        # Détecter
        image_resultat, detections = detecter_et_classifier(
            image_path, modele, CONFIG['confiance']
        )
        
        if image_resultat is not None:
            # Statistiques
            nb_vides = sum(1 for d in detections if d['etat'] == 'vide')
            nb_pleines = sum(1 for d in detections if d['etat'] == 'plein')
            
            print(f"  → {len(detections)} poubelle(s): {nb_vides} vide(s), {nb_pleines} pleine(s)")
            
            resultats_globaux.append({
                'image': image_path.name,
                'total': len(detections),
                'vides': nb_vides,
                'pleines': nb_pleines
            })
            
            # Sauvegarder
            if sauvegarder:
                sortie_path = dossier_sortie / image_path.name
                image_bgr = cv2.cvtColor(image_resultat, cv2.COLOR_RGB2BGR)
                cv2.imwrite(str(sortie_path), image_bgr)
    
    # Résumé
    print("\n" + "="*70)
    print("📊 RÉSUMÉ GLOBAL")
    print("="*70)
    
    df = pd.DataFrame(resultats_globaux)
    print(df.to_string(index=False))
    
    print("\n" + "="*70)
    print(f"Total poubelles détectées: {df['total'].sum()}")
    print(f"Total vides: {df['vides'].sum()}")
    print(f"Total pleines: {df['pleines'].sum()}")
    print("="*70)
    
    if sauvegarder:
        print(f"\n✓ Résultats sauvegardés dans: {dossier_sortie}")
        
        # Sauvegarder le CSV
        csv_path = dossier_sortie / 'resultats.csv'
        df.to_csv(csv_path, index=False)
        print(f"✓ Statistiques sauvegardées: {csv_path}")
    
    return df


# Exemple d'utilisation
# modele = charger_modele_entraine()
# resultats = detecter_dossier('chemin/vers/dossier_images', modele)

## 10. Instructions d'Utilisation

### 📋 Étapes pour utiliser ce notebook :

#### 1. **Télécharger le dataset depuis Roboflow** :
   - Exécutez la cellule 2 pour télécharger automatiquement
   - Le dataset sera téléchargé au format YOLOv9 (compatible YOLOv8)
   - Les chemins seront configurés automatiquement

#### 2. **Vérifier le dataset** :
   - Exécutez la cellule 4 pour vérifier que tout est OK
   - Vérifiez le nombre d'images dans chaque split

#### 3. **Entraîner YOLO** :
   ```python
   modele = entrainer_yolo()
   ```

#### 4. **Utiliser le modèle** :
   ```python
   # Charger le modèle
   modele = charger_modele_entraine()
   
   # Une image
   image, detections = detecter_et_classifier('image.jpg', modele)
   afficher_resultats(image, detections)
   
   # Un dossier
   detecter_dossier('dossier_images/', modele)
   
   # Webcam
   detecter_video(0, modele)
   ```

### 🎯 Avantages de cette approche :

✅ **Dataset automatique** : Téléchargement depuis Roboflow en 1 clic
✅ **Déjà annoté** : Annotations YOLO incluses
✅ **Prêt à l'emploi** : Configuration automatique
✅ **YOLO seul** : Détection + classification en une passe
✅ **Simple** : Moins de code, plus d'efficacité

### 📊 Classes détectées par votre modèle Roboflow :
Les classes seront automatiquement détectées depuis le fichier data.yaml

### 💡 Conseils :
- Vérifiez votre dataset après téléchargement
- Ajustez les hyperparamètres selon vos besoins
- Utilisez un GPU pour l'entraînement

## ✅ Résumé

Ce notebook implémente un système complet de détection et classification de poubelles avec **YOLO + Roboflow** :

### 🎯 Architecture :
- **Roboflow** : Gestion et téléchargement automatique du dataset
- **YOLOv8** : Détection ET classification en une seule étape

### ⚡ Workflow simplifié :
1. **Télécharger** le dataset depuis Roboflow (1 ligne de code)
2. **Entraîner** YOLOv8 sur vos données
3. **Utiliser** le modèle pour détecter et classifier

### 🌟 Avantages :
- ✅ Dataset professionnel depuis Roboflow
- ✅ Annotations automatiques incluses
- ✅ Configuration automatique
- ✅ YOLO fait tout en une seule étape
- ✅ Code simplifié et optimisé

### 📝 Fonctions principales (en français) :
- `telecharger_dataset_roboflow()` - Télécharge depuis Roboflow
- `verifier_dataset()` - Vérifie le dataset
- `entrainer_yolo()` - Entraîne le modèle
- `detecter_et_classifier()` - Détecte et classifie
- `detecter_video()` - Traite des vidéos
- `detecter_dossier()` - Batch processing

### 🎨 Résultats :
Les couleurs dépendront des classes de votre dataset Roboflow.

Bon entraînement ! 🚀